# 2_4 CodingGenes annotation analysis

Manuel Jara-Espejo$^{1}$\
Aboobaker lab, Department of Biology, University of Oxford

## Contents of notebook
1. Introduction
2. Reduce redundancy of annotated transcripts using CD-HIT
3. ORF identification with TransDecoder
4. Pfam and BlastP searches to enable homology-based coding region identification\
    4.1 Run Pfam search on longest ORFs \
    4.2 Run BlastP search on longest ORFs
5. Final coding region predictions
6. Select representattive transcript per gene

## Files
* Input: stringtie2_merged.gtf
* Output: 

### 1. Introduction

### 2. Reduce redundancy of annotated transcripts using CD-HIT

#### Extract transcripts sequences

In [9]:
%%bash
cd ../annotation/protein_coding_genes/stringtie2/stringtie2_merge/
#agat_sp_extract_sequences.pl -g stringtie2_merged.gtf -f /drives/ssd1/manuel/phaw/2022_analysis/phaw_gapfilling/TGS-GapCloser_anlysis/results_sambaAsm/change_scafNames/phaw_sambaAsm.scaff_seqs_editedScafNames.fa -t exon --merge -o stringtie2_merged_mRNA.fasta

#### Total number of Txs annotated in Phaw5.1
grep -c ">" ./stringtie2_merged_mRNA.fasta

131116


#### Run CH-HIT

In [10]:
%%bash 
cd ../annotation/protein_coding_genes/cdhit_analysis/
#~/software/cd-hit-v4.8.1-2019-0228/cd-hit-est -M 2000 -c 0.95 -T 8 -i $transcripts -o transcripts_cdhit_0.95.fa
head -5 transcripts_cdhit_0.95.fa.clstr

>Cluster 0
0	502nt, >MSTRG.18904.1... at +/97.21%
1	505nt, >MSTRG.18903.1... at +/96.44%
2	501nt, >MSTRG.18903.2... at +/96.41%
3	380nt, >MSTRG.19034.1... at -/97.89%


### 3. ORF identification with TransDecoder

#### 2.1 Extract the long open reading frames

In [18]:
%%bash 
cd ../annotation/protein_coding_genes/transdecoder/
#~/software/TransDecoder-TransDecoder-v5.5.0/TransDecoder.LongOrfs -S -t transcripts_cdhit_0.95.fa
#Ouput files in ./transcripts_cdhit_0.95.fa.transdecoder_dir
cd ./transcripts_cdhit_0.95.fa.transdecoder_dir
ls | grep "longest_orfs."

longest_orfs.cds
longest_orfs.cds.best_candidates.gff3
longest_orfs.cds.best_candidates.gff3.revised_starts.gff3
longest_orfs.cds.scores
longest_orfs.cds.top_500_longest
longest_orfs.cds.top_longest_5000
longest_orfs.cds.top_longest_5000.nr
longest_orfs.gff3
longest_orfs.pep


### 4. Pfam and BlastP searches to enable homology-based coding region identification

#### 4.1 Run Pfam search on longest ORFs

In [36]:
%%bash 
cd ../annotation/protein_coding_genes/pfam_search/
#The last "*" character of the peptide sequences was removed.
#sed 's/*//g' ../transdecoder/transcripts_cdhit_0.95.fa.transdecoder_dir/longest_orfs.pep > peps_filt
grep -c  ">" ./peps_filt
echo " "
#~/software/interproscan-5.52-86.0/interproscan.sh -appl Pfam -cpu 15  -goterms -iprlookup -pa -i peps_filt -f tsv
less peps_filt.tsv  | grep "GO" | head -5 | cut -f1-14

83606
 
MSTRG.3923.1.p1	1456eefee22fb663c99fd6c823ce4cc5	163	Pfam	PF02278	Polysaccharide lyase family 8, super-sandwich domain	1	143	5.6E-27	T	29-06-2022	IPR003159	Polysaccharide lyase family 8, central domain	GO:0005576|GO:0016829
MSTRG.2511.1.p1	f45ef6f317fa2f01df6b2584c25d6c93	211	Pfam	PF00018	SH3 domain	5	50	1.1E-15	T	29-06-2022	IPR001452	SH3 domain	GO:0005515
MSTRG.2511.1.p1	f45ef6f317fa2f01df6b2584c25d6c93	211	Pfam	PF00018	SH3 domain	158	203	7.7E-17	T	29-06-2022	IPR001452	SH3 domain	GO:0005515
MSTRG.28453.1.p1	2ba2bef6288d3479f8937fcd61fc7c3c	554	Pfam	PF00118	TCP-1/cpn60 chaperonin family	30	534	2.3E-153	T	29-06-2022	IPR002423	Chaperonin Cpn60/TCP-1 family	GO:0005524|GO:0016887
MSTRG.20570.1.p1	8c5e4fb56767aea8715288bcb4b5fcab	770	Pfam	PF00651	BTB/POZ domain	88	183	3.3E-22	T	29-06-2022	IPR000210	BTB/POZ domain	GO:0005515


#### 4.2 Run BlastP search on longest ORFs

In [39]:
%%bash
cd ../annotation/protein_coding_genes/blast_search/uniprot_database/
#Make blast database
# ~/software/ncbi-blast-2.12.0+/bin/makeblastdb  -in uniprot_sprot.fasta -out swissprot -dbtype prot  -parse_seqids

#Blast phaw proteome against swissprot proteome database
phaw_proteome=../../transdecoder/transcripts_cdhit_0.95.fa.transdecoder_dir/longest_orfs.pep

#~/software/ncbi-blast-2.12.0+/bin/blastp -query ${phaw_proteome} -db swissprot -max_target_seqs 1 \
#max_hsps 1 -evalue 1e-06 -num_threads 10 -outfmt '6 qseqid sseqid pident qcovs qlen slen length bitscore evalue' \
#out blast_out.txt
head -5 blast_out.txt

MSTRG.18409.1.p2	sp|P23654|NRT_DROME	37.919	56	522	846	298	186	1.69e-49
MSTRG.18410.2.p1	sp|Q9U8W7|TL5B_TACTR	45.361	37	516	316	194	167	1.68e-46
MSTRG.18412.1.p2	sp|P23654|NRT_DROME	39.207	80	263	846	227	147	5.34e-39
MSTRG.18413.1.p1	sp|Q92539|LPIN2_HUMAN	43.116	49	1088	896	552	410	5.61e-126
MSTRG.18413.2.p1	sp|Q92539|LPIN2_HUMAN	43.595	33	1531	896	523	390	6.44e-116


### 5. Final coding region predictions
Scan all ORFs for homology to known proteins and retain all such ORFs.

In [50]:
%%bash
cd ../annotation/protein_coding_genes/transdecoder/
#~/software/TransDecoder-TransDecoder-v5.5.0/TransDecoder.Predict -t transcripts_cdhit_0.95.fa.clstr \
#--retain_pfam_hits ${pfam_out} --retain_blastp_hits ${blastp_out}
#Output files prefix: transcripts_cdhit_0.95.fa.transdecoder*
ls -p | grep "transcripts_cdhit_0.95.fa.transdecoder" | grep -v /
echo " "
grep -c ">" transcripts_cdhit_0.95.fa.transdecoder.pep

transcripts_cdhit_0.95.fa.transdecoder.bed
transcripts_cdhit_0.95.fa.transdecoder.cds
transcripts_cdhit_0.95.fa.transdecoder.genome.gff3
transcripts_cdhit_0.95.fa.transdecoder.gff3
transcripts_cdhit_0.95.fa.transdecoder.pep
transdecoderComp.transcripts_cdhit_0.95.fa.transdecoder.genome.gff3.refmap
transdecoderComp.transcripts_cdhit_0.95.fa.transdecoder.genome.gff3.tmap
 
49479


##### Generate a genome-based coding region annotation file

In [6]:
%%bash
cd ../annotation/protein_coding_genes/transdecoder/
# 1. Convert stringtie_merged.gtf to .gff3
#~/software/TransDecoder-TransDecoder-v5.5.0/util/gtf_to_alignment_gff3.pl \
#../stringtie2/stringtie2_merge/stringtie2_merged.gtf > ../stringtie2/stringtie2_merge.gff3

#2. Create the coding-based annotation
#~/software/TransDecoder-TransDecoder-v5.5.0/util/cdna_alignment_orf_to_genome_orf.pl \
#transcripts_cdhit_0.95.fa.transdecoder.gff3 \
#../stringtie2/stringtie2_merge.gff3 \
#../cdhit_analysis/transcripts_cdhit_0.95.fa > transcripts_cdhit_0.95.fa.transdecoder.genome.gff3
head -5 transcripts_cdhit_0.95.fa.transdecoder.genome.gff3

Scaffold_1137_HRSCAF_3071	transdecoder	gene	16407328	16433766	.	-	.	ID=MSTRG.1000;Name=ORF%20type%3Acomplete%20len%3A117%20%28%2B%29%2Cscore%3D7.78%2Csp%7CQ1LXS2%7CPSMG2_DANRE%7C41.573%7C
Scaffold_1137_HRSCAF_3071	transdecoder	mRNA	16407328	16433766	.	-	.	ID=MSTRG.1000.1.p1;Parent=MSTRG.1000;Name=ORF%20type%3Acomplete%20len%3A117%20%28%2B%29%2Cscore%3D7.78%2Csp%7CQ1LXS2%7CPSMG2_DANRE%7C41.573%7C
Scaffold_1137_HRSCAF_3071	transdecoder	five_prime_UTR	16433460	16433766	.	-	.	ID=MSTRG.1000.1.p1.utr5p1;Parent=MSTRG.1000.1.p1
Scaffold_1137_HRSCAF_3071	transdecoder	exon	16433400	16433766	.	-	.	ID=MSTRG.1000.1.p1.exon1;Parent=MSTRG.1000.1.p1
Scaffold_1137_HRSCAF_3071	transdecoder	CDS	16433400	16433459	.	-	0	ID=cds.MSTRG.1000.1.p1;Parent=MSTRG.1000.1.p1


### 6. Select representative transcript per gene

#### 6.1 Reduce peptide redundancy using CD-HIT

In [14]:
%%bash 
cd ../annotation/protein_coding_genes/cdhit_analysis/
#~/software/cd-hit-v4.8.1-2019-0228/cd-hit -c 0.99 -T 8 -i ../transdecoder/transcripts_cdhit_0.95.fa.transdecoder.pep -o peptides.cdhit_0.99.fa
head -5 peptides.cdhit_0.99.fa.clstr

>Cluster 0
0	17390aa, >MSTRG.32169.9.p1... *
>Cluster 1
0	16406aa, >MSTRG.32169.17.p1... *
>Cluster 2


#### 6.2 Extract longest peptide ID

##### Convert peptides obtained using Transdecoder + CD-HIT tools to tbl format

In [26]:
%%bash 
#cd ../annotation/protein_coding_genes/transdecoder/unigenes/
#/drives/ssd1/manuel/FastaToTbl.sh ../../cdhit_analysis/peptides.cdhit_0.99.fa > peptides.cdhit_0.99.fa.tbl

#### 6.2 Select the longest peptide ID

In [2]:
%%bash
cd ../annotation/protein_coding_genes/transdecoder/unigenes/
#ruby get-longest-cds.rb > peptides.cdhit_0.99.fa_filter1.tbl
head -5 peptides.cdhit_0.99.fa_filter1.tbl
#cut -f3 peptides.cdhit_0.99.fa_filter1.tbl > longest_peps_id.txt
echo ""
#Longest peptide ID
head -5 longest_peps_id.txt

gene_id	transcript_id	longest_pep	pep_ln
MSTRG.1000	MSTRG.1000.1	MSTRG.1000.1.p1	117
MSTRG.10000	MSTRG.10000.1	MSTRG.10000.1.p1	509
MSTRG.10001	MSTRG.10001.1	MSTRG.10001.1.p1	128
MSTRG.10003	MSTRG.10003.2	MSTRG.10003.2.p1	1252

MSTRG.1000.1.p1
MSTRG.10000.1.p1
MSTRG.10001.1.p1
MSTRG.10003.2.p1
MSTRG.10004.1.p1


#### 6.3 Filter Transdecoder ouputs based on the longest pep id:

In [4]:
%%bash 
cd ../annotation/protein_coding_genes/transdecoder/unigenes/
#Convert .pep and .cds files to tbl format
/drives/ssd1/manuel/FastaToTbl.sh ../transcripts_cdhit_0.95.fa.transdecoder.pep > transcripts_cdhit_0.95.fa.transdecoder.pep.tbl
/drives/ssd1/manuel/FastaToTbl.sh ../transcripts_cdhit_0.95.fa.transdecoder.cds > transcripts_cdhit_0.95.fa.transdecoder.cds.tbl
	
grep -Fwf longest_peps_id.txt transcripts_cdhit_0.95.fa.transdecoder.pep.tbl > phaw_unigenes.pep.tbl
grep -Fwf longest_peps_id.txt transcripts_cdhit_0.95.fa.transdecoder.cds.tbl > phaw_unigenes.cds.tbl

#Convert tbl files to fasta format 

/drives/ssd1/manuel/TblToFasta.sh phaw_unigenes.pep.tbl > phaw_unigenes.pep
/drives/ssd1/manuel/TblToFasta.sh phaw_unigenes.cds.tbl > phaw_unigenes.cds

#Remove intermediate tbl files
rm ./*.tbl